<a href="https://colab.research.google.com/github/ENGRBertoli/docs/blob/main/primeiro_site_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import matplotlib.pyplot as plt
import pickle
import os

# Caminho do arquivo para salvar os dados
DATA_FILE = "balancas_data.pkl"

# Função para carregar os dados
def carregar_dados():
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, "rb") as file:
            return pickle.load(file)
    return []

# Função para salvar os dados
def salvar_dados(dados):
    with open(DATA_FILE, "wb") as file:
        pickle.dump(dados, file)

# Lista para armazenar as balanças (carrega do arquivo)
balancas = carregar_dados()

# Função para exibir gráfico de barras horizontais
def Grafico_Barra_Horizontal(Rotulos, Quantias, Largura=8, Altura=5, Titulo_Grafico='Gráfico de Barra', Titulo_x='Quantidade', Titulo_y='Modelos'):
    fig, ax = plt.subplots(figsize=(Largura, Altura))
    bars = ax.barh(Rotulos, Quantias, color='skyblue')
    ax.set_ylabel(Titulo_y)
    ax.set_xlabel(Titulo_x)
    ax.set_title(Titulo_Grafico)
    for bar, value in zip(bars, Quantias):
        ax.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{value}', ha='left', va='center')
    st.pyplot(fig)

# Função para obter a cor do status
def obter_cor_status(status):
    cores = {
        "Em produção": "orange",
        "Indisponível": "red",
        "Disponível para venda": "green",
        "Vendido": "blue"
    }
    return cores.get(status, "black")

# Função principal do app
def main():
    st.set_page_config(
        page_title="Painel de Gestão de Balanças",
        page_icon="⚖️",
        layout="wide",
        initial_sidebar_state="expanded"
    )

    st.title("Painel de Gestão de Fabricação de Balanças")

    # Menu lateral
    menu = st.sidebar.radio("Escolha uma opção:", ["Adicionar Balança", "Listar Balanças", "Gráficos"])

    if menu == "Adicionar Balança":
        st.subheader("Adicionar uma nova balança")
        modelo = st.text_input("Modelo da balança:")
        quantidade = st.number_input("Quantidade:", min_value=0, step=1)
        if st.button("Adicionar"):
            nova_balanca = {"modelo": modelo, "quantidade": quantidade, "status": "Em produção"}
            balancas.append(nova_balanca)
            salvar_dados(balancas)  # Salva os dados no arquivo
            st.success(f"Balança '{modelo}' adicionada com quantidade {quantidade}.")

    elif menu == "Listar Balanças":
        st.subheader("Balanças cadastradas")
        if not balancas:
            st.warning("Nenhuma balança cadastrada.")
        else:
            for idx, balanca in enumerate(balancas):
                cor = obter_cor_status(balanca["status"])
                st.markdown(
                    f"""
                    <div style='background-color: {cor}; padding: 10px; border-radius: 5px; margin-bottom: 5px;'>
                        <strong>Modelo:</strong> {balanca['modelo']} <br>
                        <strong>Quantidade:</strong> {balanca['quantidade']} <br>
                        <strong>Status:</strong> {balanca['status']}
                    </div>
                    """,
                    unsafe_allow_html=True
                )

                # Opção de alterar status
                novo_status = st.selectbox(
                    f"Alterar status do modelo '{balanca['modelo']}'",
                    ["Em produção", "Indisponível", "Disponível para venda", "Vendido"],
                    index=["Em produção", "Indisponível", "Disponível para venda", "Vendido"].index(balanca["status"]),
                    key=f"status_{idx}"
                )
                if novo_status != balanca["status"]:
                    balanca["status"] = novo_status
                    salvar_dados(balancas)  # Salva alterações
                    st.success(f"Status da balança '{balanca['modelo']}' atualizado para '{novo_status}'.")

    elif menu == "Gráficos":
        st.subheader("Gráficos das balanças")
        if not balancas:
            st.warning("Nenhuma balança cadastrada para exibir nos gráficos.")
        else:
            modelos = [b["modelo"] for b in balancas]
            quantidades = [b["quantidade"] for b in balancas]
            Grafico_Barra_Horizontal(modelos, quantidades, Largura=8, Altura=5, Titulo_Grafico="Estoque de Balanças", Titulo_x="Quantidade", Titulo_y="Modelos")

if __name__ == "__main__":
    main()

